### Assignment: Segmenting and Clustering Neighborhoods in Toronto . By CPC

#### 1) Importing Lib

In [170]:
# Importing Libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
from urllib.request import urlopen # library to open the url from which we would like to extract the data.
from bs4 import BeautifulSoup  # get the HTML structure of the page that we want to work with.
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON files
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


pd.__version__

'0.24.2'

#### 2) Extracting Data from Wikipedia and sorting

In [171]:
#Extract Data
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())
My_table = soup.find('table',{'class':'wikitable sortable'})

A=[]
B=[]
C=[]
for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3 and cells[1].find(text=True) != "Not assigned":
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

# Converting in Dataframe and solving Problem with No assigned
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df.loc[df.Neighborhood == "Not assigned\n", ['Neighborhood']] = df.loc[df.Neighborhood == "Not assigned\n", ['Borough']].values
df.shape

X = df[['PostalCode','Borough','Neighborhood']]
X.loc[X.Neighborhood == "Not assigned\n", ['Neighborhood']] = X.loc[X.Neighborhood == "Not assigned\n", ['Borough']].values


#### 3) Creating DataFrames

In [172]:
#Grouping Dataset by PostCode
Y = X.groupby(['PostalCode'])['Borough'].value_counts()
y1=[]
y2=[]
y3=[]
for i in range(len(Y)):
    y1.append(Y.index[i][0])
    y2.append(Y.index[i][1])
    
dy=pd.DataFrame(y1,columns=['PostalCode'])
dy['Borough']=y2
dy['cant_Neig']=Y.values
dy['Neighborhood'] = ""
#print(dy.shape)
#dy.head()
z1=[]
i=0
dy['Neighborhood'] = ""

##############

#Grouping Dataset by Neighborhood
Z = X.groupby(['PostalCode'])['Neighborhood'].value_counts()
z1=[]
z2=[]
for i in range(len(Z)):
    z1.append(Z.index[i][0])
    z2.append(Z.index[i][1])
dz=pd.DataFrame(z1,columns=['PostalCode'])
dz['Neighborhood']=z2
#dz.head()

#Creating the final dataFrame
z1=[]
i=0
dy['Neighborhood'] = ""
for i in range(len(dy)):
    h=0
    yuca=""
    for h in range(len(dz)):
        if dy.iloc[i,0] == dz.iloc[h,0]:#Deberia estar len(dz)
                yuca = dz.iloc[h,1]
                dy.iloc[i,3] = dy.iloc[i,3] + ', ' + yuca
                h = h+1               
dy = dy.drop(['cant_Neig'], axis = 1)

## - Result of the First part of the assignment - DataFrame with Postalcode, Borough and Neignborhood
-------------------------------------------------------------------------------------

In [173]:
print(dy.shape)
dy[0:10]

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,", Malvern, Rouge"
1,M1C,Scarborough,", Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,", Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,", Woburn"
4,M1H,Scarborough,", Cedarbrae\n"
5,M1J,Scarborough,", Scarborough Village"
6,M1K,Scarborough,", East Birchmount Park\n, Ionview, Kennedy Park"
7,M1L,Scarborough,", Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,", Cliffcrest, Cliffside, Scarborough Village W..."
9,M1N,Scarborough,", Birch Cliff, Cliffside West\n"
